In [1]:
import importlib
import os
import time
from tqdm import tqdm
import itertools
import numpy as np
import json
import mosa
import matplotlib.pyplot as plt
import pyvista as pv
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from scipy.optimize import fsolve
from numpy import random
import pandas as pd

from scipy.optimize import fsolve
from math import inf
from numpy.random import seed

In [2]:
# -------------- PART 0: CHOOSE CIRCUIT AND SET UP FOLDER --------------


# Choose circuit
circuit = "metab1"

# Import circuit config file
config = importlib.import_module(circuit)

# Define the subfolder name
folder_name = f"MOSA_{circuit}"

# Create folder if not yet exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Jump to folder
os.chdir(folder_name)

# Prompt new folder name
print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/nt625/Documents/GitHub/My5thMOSArepository_plsdontcrash/Metabolic1/MOSA_metab1


In [3]:
# -------------- PART 0b: DEFINE DYNAMICAL SYSTEM --------------


# Define number of steady states expected
numss = int(1)

In [4]:
# -------------- PART 0c: DEFINE SENSITIVITY FUNCTIONS --------------

# Define analytical sensitivity expressions
S_kE1_P_analytic = config.S_kE1_P_analytic
S_kE2_P_analytic = config.S_kE2_P_analytic
S_kE1_S2_analytic = config.S_kE1_S2_analytic
S_kE2_S2_analytic = config.S_kE2_S2_analytic
S_kE1_ATP_analytic = config.S_kE1_ATP_analytic
S_kE2_ATP_analytic = config.S_kE2_ATP_analytic

In [5]:
# -------------- PART 0d: CHOOSE SENSITIVITY FUNCTIONS --------------

# Print prompt
print("""
We have the following sensitivity functions:
0. |S_kE1_P|
1. |S_kE2_P|
2. |S_kE1_S2|
3. |S_kE2_S2|
4. |S_kE1_ATP|
5. |S_kE2_ATP|
""")

# Choose pair of functions
choice1 = int(input("Please select first option number:"))
choice2 = int(input("Please select second option number:"))

# List of sensitivity function names
sensitivity_labels = [
    "|S_kE1_P|",
    "|S_kE2_P|",
    "|S_kE1_S2|",
    "|S_kE2_S2|",
    "|S_kE1_ATP|",
    "|S_kE2_ATP|"]

# Save function names for later use
label1 = sensitivity_labels[choice1]
label2 = sensitivity_labels[choice2]

# Name for text file to records stats
output_file = f"TwoSpecies_{choice1}_and_{choice2}.csv"


We have the following sensitivity functions:
0. |S_kE1_P|
1. |S_kE2_P|
2. |S_kE1_S2|
3. |S_kE2_S2|
4. |S_kE1_ATP|
5. |S_kE2_ATP|



In [6]:
# -------------- PART 0e: CHANGING DIRECTORIES --------------

# Define the subfolder name
subfolder_name = f"MOSA_sensfuncs_{choice1}_and_{choice2}"

# Create folder if not yet exist
if not os.path.exists(subfolder_name):
    os.makedirs(subfolder_name)

# Jump to folder
os.chdir(subfolder_name)

# Prompt new folder name
print(f"Current working directory: {os.getcwd()}")

if not os.path.exists('data'):
    os.makedirs('data')

Current working directory: /Users/nt625/Documents/GitHub/My5thMOSArepository_plsdontcrash/Metabolic1/MOSA_metab1/MOSA_sensfuncs_4_and_5


In [7]:
# -------------- PART 0f: DEFINE FUNCTIONS --------------

In [8]:
# DEFINE FUNCTION THAT SOLVES FOR STEADY STATES XSS AND YSS GIVEN SOME INITIAL GUESS
def ssfinder(rS1_val, kS1_val, kS2_val, kP_val, kE1_val, kE2_val, alphaE_val, gammaE_val):

    # If we have one steady state
    if numss == 1: 

        S2ss = (1/kS2_val) * kE1_val * (kS1_val/rS1_val) * (alphaE_val/gammaE_val)
        S2ss_prime = S2ss
        Pss =  (1/kP_val) * kE1_val * (kE2_val/kS2_val) * (kS1_val/rS1_val) * (alphaE_val/gammaE_val)**2
        Pss_prime = - Pss
        ATPss = (alphaE_val/gammaE_val)*(kS1_val/rS1_val) * (kE1_val + kE2_val*(kE1_val/kS2_val)*(alphaE_val/gammaE_val))
        ATPss_prime = ATPss
        
        return S2ss_prime, Pss_prime, ATPss_prime

In [9]:
# DEFINE FUNCTION THAT RETURNS PAIR OF SENSITIVITIES
def senpair(rS1_val, kS1_val, kS2_val, kP_val, kE1_val, kE2_val, alphaE_val, gammaE_val, choice1, choice2):
    
    # Evaluate sensitivities
    S_kE1_P   = S_kE1_P_analytic()
    S_kE2_P   = S_kE2_P_analytic()
    S_kE1_S2  = S_kE1_S2_analytic()
    S_kE2_S2  = S_kE2_S2_analytic()
    S_kE1_ATP = S_kE1_ATP_analytic()
    S_kE2_ATP = S_kE2_ATP_analytic(kS2_val, kE2_val, alphaE_val, gammaE_val)

    # Sensitivity dictionary
    sensitivities = {
        "S_kE1_P": S_kE1_P,
        "S_kE2_P": S_kE2_P,
        "S_kE1_S2": S_kE1_S2,
        "S_kE2_S2": S_kE2_S2,
        "S_kE1_ATP": S_kE1_ATP,
        "S_kE2_ATP": S_kE2_ATP}

    # Map indices to keys
    labels = {
        0: "S_kE1_P",
        1: "S_kE2_P",
        2: "S_kE1_S2",
        3: "S_kE2_S2",
        4: "S_kE1_ATP",
        5: "S_kE2_ATP"}

    # Return values of the two sensitivities of interest
    return sensitivities[labels[choice1]], sensitivities[labels[choice2]]

In [10]:
# DEFINE OBJECTIVE FUNCTION TO ANNEAL
def fobj(solution):

    print("solution")
    print(solution)
	
	# Update parameter set
    rS1_val = solution["rS1"]
    kS1_val = solution["kS1"]
    kS2_val = solution["kS2"]
    kP_val = solution["kP"]
    kE1_val = solution["kE1"]
    kE2_val = solution["kE2"]
    alphaE_val = solution["alphaE"]
    gammaE_val = solution["gammaE"]

    # # Create an empty numpy array
    # array1 = np.array([])
    # array2 = np.array([])
    # array3 = np.array([])

    # Find steady states and store.
    S2ss_prime, Pss_prime, ATPss_prime = ssfinder(rS1_val, kS1_val, kS2_val, kP_val, kE1_val, kE2_val, alphaE_val, gammaE_val)
    # array1 = np.append(array1, S2ss_prime)
    # array2 = np.append(array2, Pss_prime)
    # array3 = np.append(array3, ATPss_prime)

    # print("arrays")
    # print(array1)
    # print(array2)
    # print(array3)

    # Get sensitivity pair
    sens1, sens2 = senpair(rS1_val, kS1_val, kS2_val, kP_val, kE1_val, kE2_val, alphaE_val, gammaE_val, choice1, choice2)
    ans1 = float(sens1)
    ans2 = float(sens2)

    return S2ss_prime, Pss_prime, ATPss_prime, ans1, ans2

In [11]:
# -------------- PART 1: GAUGING MOSA PARAMETERS --------------
    
# Sample rS1 values
rS1_min = 0.01
rS1_max = 5
rS1_sampsize = 5
rS1_samps = np.linspace(rS1_min, rS1_max, rS1_sampsize)

# Sample kS1 values
kS1_min = 0.01
kS1_max = 5
kS1_sampsize = 5
kS1_samps = np.linspace(kS1_min, kS1_max, kS1_sampsize)

# Sample kS2 values
kS2_min = 0.01
kS2_max = 5
kS2_sampsize = 5
kS2_samps = np.linspace(kS2_min, kS2_max, kS2_sampsize)

# Sample kP values
kP_min = 0.01
kP_max = 5
kP_sampsize = 5
kP_samps = np.linspace(kP_min, kP_max, kP_sampsize)

# Sample kE1 values
kE1_min = 0.01
kE1_max = 5
kE1_sampsize = 5
kE1_samps = np.linspace(kE1_min, kE1_max, kE1_sampsize)

# Sample kE2 values
kE2_min = 0.01
kE2_max = 5
kE2_sampsize = 5
kE2_samps = np.linspace(kE2_min, kE2_max, kE2_sampsize)

# Sample alphaE values
alphaE_min = 0.01
alphaE_max = 5
alphaE_sampsize = 5
alphaE_samps = np.linspace(alphaE_min, alphaE_max, alphaE_sampsize)

# Sample gammaE values
gammaE_min = 0.01
gammaE_max = 5
gammaE_sampsize = 5
gammaE_samps = np.linspace(gammaE_min, gammaE_max, gammaE_sampsize)


# Create empty arrays to store ...

# ... observables
S2ss_prime_samps = np.array([])
Pss_prime_samps = np.array([])
ATPss_prime_samps = np.array([])
# ... sensitivities
sens1_samps = np.array([])
sens2_samps = np.array([])

In [12]:
# WITH LOADING BAR 
# Compute the total number of iterations for tqdm
total_iterations = rS1_sampsize * kS1_sampsize * kS2_sampsize * kP_sampsize * kE1_sampsize * kE2_sampsize * alphaE_sampsize * gammaE_sampsize
# Loop over every combination of parameters with a progress bar
for i, j, k, l, m, n, o, p in tqdm(itertools.product(rS1_samps, kS1_samps, kS2_samps, kP_samps, kE1_samps, kE2_samps, alphaE_samps, gammaE_samps), total=total_iterations, desc="Gauging energies:"):
    
    # Get steady states and store
        S2ss_prime, Pss_prime, ATPss_prime = ssfinder(i, j, k, l, m, n, o, p)
        
        S2ss_prime_samps = np.append(S2ss_prime_samps, S2ss_prime)
        Pss_prime_samps = np.append(Pss_prime_samps, Pss_prime)
        ATPss_prime_samps = np.append(ATPss_prime_samps, ATPss_prime)

        # Get sensitivities and store
        sens1, sens2 = senpair(i, j, k, l, m, n, o, p, choice1, choice2)
        sens1_samps = np.append(sens1_samps, sens1)
        sens2_samps = np.append(sens2_samps, sens2)

Gauging energies:: 100%|██████████| 390625/390625 [02:48<00:00, 2314.38it/s] 


In [13]:
# Get min and max of each sensitivity and print

S2ss_prime_samps_min = np.nanmin(S2ss_prime_samps)
Pss_prime_samps_min = np.nanmin(Pss_prime_samps)
ATPss_prime_samps_min = np.nanmin(ATPss_prime_samps)
sens1_samps_min = np.nanmin(sens1_samps)
sens2_samps_min = np.nanmin(sens2_samps)

S2ss_prime_samps_max = np.nanmax(S2ss_prime_samps)
Pss_prime_samps_max = np.nanmax(Pss_prime_samps)
ATPss_prime_samps_max = np.nanmax(ATPss_prime_samps)
sens1_samps_max = np.nanmax(sens1_samps)
sens2_samps_max = np.nanmax(sens2_samps)

print(S2ss_prime_samps_min)
print(S2ss_prime_samps_max)
print(Pss_prime_samps_min)
print(Pss_prime_samps_max)
print(ATPss_prime_samps_min)
print(ATPss_prime_samps_max)
print(sens1_samps_min)
print(sens1_samps_max)
print(sens2_samps_min)
print(sens2_samps_max)


8e-09
125000000.0
-31250000000000.0
-3.2e-14
4.000016e-08
312501250000.0
1.0
1.0
3.999984000064e-06
0.999996000016


In [14]:
# Get MOSA energies

deltaE_S2ss_prime = abs(S2ss_prime_samps_max - S2ss_prime_samps_min)
deltaE_Pss_prime = abs(Pss_prime_samps_max - Pss_prime_samps_min)
deltaE_ATPss_prime = abs(ATPss_prime_samps_max - ATPss_prime_samps_min)

deltaE_sens1 = abs(sens1_samps_max - sens1_samps_min)
deltaE_sens2 = abs(sens2_samps_max - sens2_samps_min)

deltaE = np.linalg.norm([deltaE_sens1, deltaE_sens2])

deltaE

0.999992000032

In [15]:
# Get hot temperature
print("Now setting up hot run...")
probability_hot = float(0.9)
temp_hot = deltaE / np.log(1/probability_hot)

# Get cold temperature
print("Now setting up cold run...")
probability_cold = float(0.01)
temp_cold = deltaE / np.log(1/probability_cold)

print(temp_hot)
print(temp_cold)

Now setting up hot run...
Now setting up cold run...
9.491145651560968
0.21714550378064698


In [16]:
# DUMMY

In [17]:
# Set random number generator to 0
seed(0)

In [32]:
# Initialisation of MOSA
opt = mosa.Anneal()
opt.archive_size = 100
opt.maximum_archive_rejections = opt.archive_size * 2
opt.population = {"rS1": (rS1_min, rS1_max),"rS2": (rS1_min, rS1_max),"rS3": (rS1_min, rS1_max), "kS1": (kS1_min, kS1_max), "kS2": (kS2_min, kS2_max), "kP": (kP_min, kP_max), "kE1": (kE1_min, kE1_max), "kE2": (kE2_min, kE2_max), "alphaE": (alphaE_min, alphaE_max), "gammaE": (gammaE_min, gammaE_max)}


--------------------------------------------------
    MULTI-OBJECTIVE SIMULATED ANNEALING (MOSA)    
--------------------------------------------------
         Developed by Prof. Roberto Gomes         
   Universidade Federal do ABC (UFABC), Brazil    




In [33]:
# Hot run options
opt.initial_temperature = temp_hot
opt.number_of_iterations = 100
opt.number_of_temperatures = int(np.ceil((temp_hot-temp_cold)/opt.temperature_decrease_factor)) # want to get close to Tcold
opt.temperature_decrease_factor = 0.9
opt.number_of_solution_elements = {"rS1": 1,"rS2": 1,"rS3": 1, "kS1": 1, "kS2": 1, "kP": 1, "kE1": 1, "kE2": 1, "alphaE": 1, "gammaE": 1}
step_scaling = 1/opt.number_of_iterations
opt.mc_step_size = {"rS1": abs(rS1_min - rS1_max)*step_scaling,"rS2": abs(rS1_min - rS1_max)*step_scaling,"rS3": abs(rS1_min - rS1_max)*step_scaling, "kS1": abs(kS1_min - kS1_max)*step_scaling, "kS2": abs(kS2_min - kS2_max)*step_scaling, "kP": abs(kP_min - kP_max)*step_scaling, "kE1": abs(kE1_min - kE1_max)*step_scaling, "kE2": abs(kE2_min - kE2_max)*step_scaling, "alphaE": abs(alphaE_min - alphaE_max)*step_scaling, "gammaE": abs(gammaE_min - gammaE_max)*step_scaling}


# Hot run
start_time = time.time()
hotrun_stoppingtemp = opt.evolve(fobj)
print(f"Hot run time: {time.time() - start_time} seconds")

--- BEGIN: Evolving a solution ---

Looking for a solution in the checkpoint file...
No checkpoint file!
Done!
Trying to load the archive from file archive.json...
File archive.json not found! Initializing an empty archive...
Done!
------
Keys in the population/solution dictionaries:
    ['rS1']:
        Number of elements in the solution: 1
        Continuous sample space
        Boundaries: (0.010000,5.000000)
        Selection weight of this key: 1.000000
        Weight of 'change value' trial move: 1.000000
        Solution sorted after trial move: False
        Maximum step size to choose a new value in the solution: 0.049900
    ['rS2']:
        Number of elements in the solution: 1
        Continuous sample space
        Boundaries: (0.010000,5.000000)
        Selection weight of this key: 1.000000
        Weight of 'change value' trial move: 1.000000
        Solution sorted after trial move: False
        Maximum step size to choose a new value in the solution: 0.049900
    ['r

In [ ]:




# Cold run options
opt.initial_temperature = temp_cold
opt.number_of_iterations = 100
opt.number_of_temperatures = 100
opt.temperature_decrease_factor = 0.9
step_scaling = 0.1/opt.number_of_iterations
opt.mc_step_size = {"rS1": abs(rS1_min - rS1_max)*step_scaling, "kS1": abs(kS1_min - kS1_max)*step_scaling, "kS2": abs(kS2_min - kS2_max)*step_scaling, "kP": abs(kP_min - kP_max)*step_scaling, "kE1": abs(kE1_min - kE1_max)*step_scaling, "kE2": abs(kE2_min - kE2_max)*step_scaling, "alphaE": abs(alphaE_min - alphaE_max)*step_scaling, "gammaE": abs(gammaE_min - gammaE_max)*step_scaling}



# Cold run
start_time = time.time()
coldrun_stoppingtemp = opt.evolve(fobj)
print(f"Cold run time: {time.time() - start_time} seconds")


# Output 
start_time = time.time()
pruned = opt.prunedominated()
opt.plotfront(pruned)
print(f"Output time: {time.time() - start_time} seconds")